# Agent Debates with Tools

This example shows how to simulate multi-agent dialogues where agents have access to tools.

## Import LangChain related modules 

In [1]:
from typing import List, Dict, Callable
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.prompts.prompt import PromptTemplate
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)
import os
import json
import re
from langchain import Anthropic
from langchain.chat_models import ChatAnthropic
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    HumanMessage,
)
from typing import List
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.agents import tool
from pydantic import BaseModel, Field
from solcx import install_solc, compile_source
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, BaseMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
from typing import List
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.schema import SystemMessage, BaseMessage, HumanMessage, AIMessage
from utils.tools import compile_solidity, SolidityCompilerInput



load_dotenv()


def get_secret(name):
    if os.environ[name]:
        return os.environ[name]
    return os.getenv(name)


Anthropic.anthropic_api_key = get_secret('ANTHROPIC_API_KEY')

with open('data/e2e_suggest_test_from_tests_example.json', 'r') as f_in:
    example = json.load(f_in)

## Import modules related to tools

In [2]:
from langchain.agents import Tool
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents import load_tools

## `DialogueAgent` and `DialogueSimulator` classes
We will use the same `DialogueAgent` and `DialogueSimulator` classes defined in [Multi-Player Authoritarian Speaker Selection](https://python.langchain.com/en/latest/use_cases/agent_simulations/multiagent_authoritarian.html).

In [3]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()
        
    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        
    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

## `DialogueAgentWithTools` class
We define a `DialogueAgentWithTools` class that augments `DialogueAgent` to use tools.

In [4]:
class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = tools

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        agent_chain = initialize_agent(
            self.tools, 
            self.model, 
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, 
            verbose=True, 
            memory=ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        )
        message = AIMessage(content=agent_chain.run(
            input="\n".join([
                self.system_message.content] + \
                self.message_history + \
                [self.prefix])))
        
        return message.content

## Define roles and topic

In [88]:
example['complete_example']['contract_code'] = """pragma solidity ^0.8.17;

contract SimpleStorage {
    // State variable to store a number
    uint public num;

    // You need to send a transaction to write to a state variable.
    function set(uint _num) public {
        num = _num;
    }

    // You can read from a state variable without sending a transaction.
    function get() public view returns (uint) {
        return num;
    }
}"""
example['complete_example']['test_contract_code'] = ""
example['complete_example']['function'] = "set"
example['complete_example_answer'] = """pragma solidity ^0.8.17;

import "truffle/Assert.sol";
import "truffle/DeployedAddresses.sol";
import "../contracts/SimpleStorage.sol";

contract TestSimpleStorage {
    function testSet() public {
        // Get the deployed contract instance
        SimpleStorage simpleStorage = SimpleStorage(DeployedAddresses.SimpleStorage());

        // Set a value
        uint expectedValue = 42;
        simpleStorage.set(expectedValue);

        // Retrieve the value
        uint storedValue = simpleStorage.get();

        // Assert that the retrieved value matches the expected value
        Assert.equal(storedValue, expectedValue, "Stored value should match the expected value");
    }
}
"""



example['incomplete_example']['contract_code'] = """// SPDX-License-Identifier: MIT
pragma solidity ^0.8.17;

contract Loop {
    function loop() public {
        // for loop
        for (uint i = 0; i < 10; i++) {
            if (i == 3) {
                // Skip to next iteration with continue
                continue;
            }
            if (i == 5) {
                // Exit loop with break
                break;
            }
        }

        // while loop
        uint j;
        while (j < 10) {
            j++;
        }
    }
}
"""
example['incomplete_example']['test_contract_code'] = ""
example['incomplete_example']['function'] = "loop"
assistant_inception_prompt = example['assistant_inception_prompt']
user_inception_prompt = example['user_inception_prompt']
assistant_role_name = "Web3 Security Expert"
user_role_name = "Solidity Developer"
task = example['user_task'].format(**example['incomplete_example']) 
            
names = {
    user_role_name: [
        'solidity compiler', 
    ],
    assistant_role_name: [
        'solidity compiler', 
    ],
}
word_limit = 500 # word limit for task brainstorming
tools = [
            Tool(
                func=compile_solidity,
                name="solidity compiler",
                description="Can compile solidity code. Accepts ONLY solidity code",
                args_schema=SolidityCompilerInput,
            ),
        ]
tool_names = [t.name for t in tools]

In [71]:
task

'\nHere is the smart contract code to be tested: // SPDX-License-Identifier: MIT\npragma solidity ^0.8.17;\n\ncontract Loop {\n    function loop() public {\n        // for loop\n        for (uint i = 0; i < 10; i++) {\n            if (i == 3) {\n                // Skip to next iteration with continue\n                continue;\n            }\n            if (i == 5) {\n                // Exit loop with break\n                break;\n            }\n        }\n\n        // while loop\n        uint j;\n        while (j < 10) {\n            j++;\n        }\n    }\n}\n\nHere is the test contract in Solidity: \nHere is the specific function you need to test in the new end-to-end test: loop\nRecommend a new end-to-end test wrapped in a Solidity function with no other comments or explanation.'

In [67]:



def get_sys_msgs(assistant_role_name: str, user_role_name: str, task: str):
    assistant_sys_template = SystemMessagePromptTemplate.from_template(template=assistant_inception_prompt)
    assistant_sys_msg = \
        assistant_sys_template.format_messages(assistant_role_name=assistant_role_name,
                                               user_role_name=user_role_name,
                                               task=task)[0]

    user_sys_template = SystemMessagePromptTemplate.from_template(template=user_inception_prompt)
    user_sys_msg = \
        user_sys_template.format_messages(assistant_role_name=assistant_role_name,
                                          user_role_name=user_role_name,
                                          task=task)[0]

    return assistant_sys_msg, user_sys_msg

task_specifier_prompt = (
            """Here is a task that {assistant_role_name} will help {user_role_name} to complete: {task}.
            Please make it more specific. Be creative and imaginative.
            Please reply with the specified task in {word_limit} words or less. Do not add anything else."""
        )
task_specifier_template = HumanMessagePromptTemplate.from_template(template=task_specifier_prompt)
task_specifier_msg = task_specifier_template.format_messages(assistant_role_name=assistant_role_name,
                                                             user_role_name=user_role_name,
                                                             task=task, word_limit=word_limit)[0]
assistant_sys_msg, user_sys_msg = get_sys_msgs(assistant_role_name, user_role_name,
                                                    task_specifier_msg.content)

In [72]:
assistant_sys_msg

SystemMessage(content='Never forget you are a Web3 Security Expert and I am a Solidity Developer. Never flip roles! Never instruct me!\n            We share a common interest in collaborating to successfully complete a task.\n            You must help me to complete the task.\n            Here is the task: Here is a task that Web3 Security Expert will help Solidity Developer to complete: \nHere is the smart contract code to be tested: // SPDX-License-Identifier: MIT\npragma solidity ^0.8.17;\n\ncontract Loop {\n    function loop() public {\n        // for loop\n        for (uint i = 0; i < 10; i++) {\n            if (i == 3) {\n                // Skip to next iteration with continue\n                continue;\n            }\n            if (i == 5) {\n                // Exit loop with break\n                break;\n            }\n        }\n\n        // while loop\n        uint j;\n        while (j < 10) {\n            j++;\n        }\n    }\n}\n\nHere is the test contract in Solidity:

In [74]:
agent_descriptions

{'Web3 Security Expert': 'Never forget you are a Web3 Security Expert and I am a Solidity Developer. Never flip roles! Never instruct me!\n            We share a common interest in collaborating to successfully complete a task.\n            You must help me to complete the task.\n            Here is the task: \nHere is the smart contract code to be tested: // SPDX-License-Identifier: MIT\npragma solidity ^0.8.17;\n\ncontract Loop {\n    function loop() public {\n        // for loop\n        for (uint i = 0; i < 10; i++) {\n            if (i == 3) {\n                // Skip to next iteration with continue\n                continue;\n            }\n            if (i == 5) {\n                // Exit loop with break\n                break;\n            }\n        }\n\n        // while loop\n        uint j;\n        while (j < 10) {\n            j++;\n        }\n    }\n}\n\nHere is the test contract in Solidity: \nHere is the specific function you need to test in the new end-to-end test: lo

## Ask an LLM to add detail to the topic description

In [70]:
assistant_description = assistant_inception_prompt.format(
    assistant_role_name=assistant_role_name, user_role_name=user_role_name, task=task)
user_description = user_inception_prompt.format(
    assistant_role_name=assistant_role_name, user_role_name=user_role_name, task=task)
        
agent_descriptions = {assistant_role_name: assistant_description,
                      user_role_name: user_description}

In [75]:
for name, description in agent_descriptions.items():
    print(description)

Never forget you are a Web3 Security Expert and I am a Solidity Developer. Never flip roles! Never instruct me!
            We share a common interest in collaborating to successfully complete a task.
            You must help me to complete the task.
            Here is the task: 
Here is the smart contract code to be tested: // SPDX-License-Identifier: MIT
pragma solidity ^0.8.17;

contract Loop {
    function loop() public {
        // for loop
        for (uint i = 0; i < 10; i++) {
            if (i == 3) {
                // Skip to next iteration with continue
                continue;
            }
            if (i == 5) {
                // Exit loop with break
                break;
            }
        }

        // while loop
        uint j;
        while (j < 10) {
            j++;
        }
    }
}

Here is the test contract in Solidity: 
Here is the specific function you need to test in the new end-to-end test: loop
Recommend a new end-to-end test wrapped in a Solidity

## Generate system messages

In [76]:
agent_system_messages = {assistant_role_name: assistant_sys_msg,
                      user_role_name: user_sys_msg}

In [77]:
for name, system_message in agent_system_messages.items():
    print(name)
    print(system_message)

Web3 Security Expert
content='Never forget you are a Web3 Security Expert and I am a Solidity Developer. Never flip roles! Never instruct me!\n            We share a common interest in collaborating to successfully complete a task.\n            You must help me to complete the task.\n            Here is the task: Here is a task that Web3 Security Expert will help Solidity Developer to complete: \nHere is the smart contract code to be tested: // SPDX-License-Identifier: MIT\npragma solidity ^0.8.17;\n\ncontract Loop {\n    function loop() public {\n        // for loop\n        for (uint i = 0; i < 10; i++) {\n            if (i == 3) {\n                // Skip to next iteration with continue\n                continue;\n            }\n            if (i == 5) {\n                // Exit loop with break\n                break;\n            }\n        }\n\n        // while loop\n        uint j;\n        while (j < 10) {\n            j++;\n        }\n    }\n}\n\nHere is the test contract in So

## Main Loop

In [90]:
# we set `top_k_results`=2 as part of the `tool_kwargs` to prevent results from overflowing the context limit
agents = [DialogueAgentWithTools(name=name,
                     system_message=system_message, 
                     model=ChatOpenAI(
                         model_name='gpt-3.5-turbo',
                         temperature=0.2),
                     tools=tools,
                     tool_names=tools,
                     top_k_results=2,
                                ) for (name, tools), system_message in zip(names.items(), agent_system_messages.values())]

ValueError: Got unknown tool solidity compiler

In [33]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = (step) % len(agents)
    return idx

In [34]:
max_iters = 6
n = 0

simulator = DialogueSimulator(
    agents=agents,
    selection_function=select_next_speaker
)
simulator.reset()
simulator.inject('Moderator', specified_topic)
print(f"(Moderator): {specified_topic}")
print('\n')

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print('\n')
    n += 1

NameError: name 'agents' is not defined